In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.models as models

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_data = datasets.ImageFolder('da/images/train', transform=transform)
val_data = datasets.ImageFolder('da/images/val', transform=transform)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)

# Define AlexNet
model = models.alexnet(weights='AlexNet_Weights.DEFAULT')
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # 2 classes: normal and pneumonia
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [2]:
# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

Epoch 1/25, Loss: 0.542021114230156
Epoch 2/25, Loss: 0.4628302589058876
Epoch 3/25, Loss: 0.4357836216688156
Epoch 4/25, Loss: 0.42069445714354514
Epoch 5/25, Loss: 0.40420722037553786
Epoch 6/25, Loss: 0.3865345712006092
Epoch 7/25, Loss: 0.37157567739486697
Epoch 8/25, Loss: 0.3636331923305988
Epoch 9/25, Loss: 0.3520447562634945
Epoch 10/25, Loss: 0.3302772767841816
Epoch 11/25, Loss: 0.3166525182127953
Epoch 12/25, Loss: 0.3000302381813526
Epoch 13/25, Loss: 0.2759896965324879
Epoch 14/25, Loss: 0.26686863899230956
Epoch 15/25, Loss: 0.2465308404713869
Epoch 16/25, Loss: 0.22688486926257612
Epoch 17/25, Loss: 0.2062791299074888
Epoch 18/25, Loss: 0.19190244555473326
Epoch 19/25, Loss: 0.16348863303661346
Epoch 20/25, Loss: 0.16293494068086148
Epoch 21/25, Loss: 0.11654743369668723
Epoch 22/25, Loss: 0.11299249716103077
Epoch 23/25, Loss: 0.12006410330533981
Epoch 24/25, Loss: 0.08016463853418827
Epoch 25/25, Loss: 0.08325706865638495


In [3]:
torch.save(model.state_dict(), 'mo/alex.pth')
model.load_state_dict(torch.load('mo/alex.pth'))

<All keys matched successfully>

In [4]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

# Evaluation
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy*100:.4f}')

conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

recall = recall_score(y_true, y_pred)
print(f'Recall: {recall*100:.4f}')

precision = precision_score(y_true, y_pred)
print(f'Precision: {precision*100:.4f}')

f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1*100:.4f}')

Accuracy: 82.7500
Confusion Matrix:
[[604 196]
 [ 80 720]]
Recall: 90.0000
Precision: 78.6026
F1 Score: 83.9161
